<a href="https://colab.research.google.com/github/Flychuban/Space-Soup/blob/main/TextSummarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 92.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.6 MB/s eta 0:00:00


In [2]:
import requests
from transformers import pipeline
from bs4 import BeautifulSoup
import re

In [3]:
# Load summarization model from HuggingFace
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


Get text from web page

In [71]:
def search_urls_space_news():
  URL = "https://www.google.com/search?q=nasa+news&tbm=nws"
  r = requests.get(URL)
  soup = BeautifulSoup(r.text, 'html.parser')
  atags = soup.find_all(['a'])
  hrefs = [link['href'] for link in atags]
  return hrefs


In [72]:
hrefs = search_urls_space_news()

In [73]:
exclude_list = ['maps', 'policies', 'preferences', 'accounts', 'support']

In [74]:
def strip_unwanted_urls(urls, exclude_list):
    val = []
    for url in urls:
        if 'https://' in url and not any(exclude_word in url for exclude_word in exclude_list):
            res = re.findall(r'(https?://\S+)', url)[0].split('&')[0]
            val.append(res)
    return list(set(val))

In [75]:
cleaned_urls = strip_unwanted_urls(hrefs, exclude_list)
cleaned_urls

['https://www.nbcnews.com/science/space/first-nasa-returns-asteroid-samples-earth-rcna111474',
 'https://www.cbsnews.com/video/nasa-mission-to-asteroid-return-to-earth/',
 'https://www.nasa.gov/feature/goddard/2023/nasa-s-webb-finds-carbon-source-on-surface-of-jupiter-s-moon-europa',
 'https://www.google.com/search?q%3Dnasa%2Bnews%26tbm%3Dnws%26pccc%3D1',
 'https://webbtelescope.org/contents/news-releases/2023/news-2023-113',
 'https://www.nature.com/articles/d41586-023-02893-y',
 'https://spaceflightnow.com/2023/09/22/nasa-astronaut-looks-forward-to-family-hugs-peace-and-quiet-after-yearlong-flight/',
 'https://mashable.com/video/nasa-spacecraft-solar-ejection-cme',
 'https://www.theguardian.com/science/live/2023/sep/24/bennu-asteroid-sample-nasa-osiris-rex-mission-earth-updates',
 'https://www.nbcnews.com/video/asteroid-sample-collected-by-nasa-returns-to-earth-193654853618',
 'https://www.nytimes.com/2023/09/24/science/nasa-osiris-rex-asteroid-sample-landing.html',
 'https://www.cbs

In [188]:
def get_text_from_urls(urls):
  articles = []
  titles = []
  for url in urls:
    headers = {'User-Agent': 'Mozilla/5.0'}
    cookies = {"CONSENT": "YES+cb.20210720-07-p0.en+FX+410"}
    r = requests.get(url, headers=headers, cookies=cookies)
    if r.status_code != 200:
      continue

    soup = BeautifulSoup(r.text, 'html.parser')
    paragraphs = soup.find_all(['p'])
    text = [paragraph.text for paragraph in paragraphs]
    # words = ' '.join(text)
    article = ' '.join(text)
    article = article.rstrip()

    # Minimal number of symbols in article, prevent returning cookies or terms error
    if len(article) < 200:
      continue

    articles.append(article)
    url_h1_tags = soup.find_all(['h1'])
    url_titles = [title.text.strip() for title in url_h1_tags]
    titles.append(url_titles)

  return titles, articles

In [218]:
titles, articles = get_text_from_urls(cleaned_urls)

In [219]:
articles

[' Profile Sections tv Featured More From NBC Follow NBC News ‘I am not going anywhere’: Sen. Bob Menendez rejects calls to resign over bribery charges September 22, 2023, 4:39 PM New Jersey Gov. Phil Murphy calls on Sen. Bob Menendez to resign over bribery charges September 22, 2023, 3:07 PM Biden tells Zelenskyy that the U.S. will send long-range missiles that Ukraine has repeatedly asked for, people familiar with the discussions say September 22, 2023, 11:49 AM Autoworkers strike will expand to more GM and Stellantis plants, and the head of the UAW extends an invitation for Biden to join the picket line September 22, 2023, 11:00 AM A capsule containing precious samples from an asteroid landed safely on Earth on Sunday, the culmination of a roughly 4-billion-mile journey over the past seven years. The asteroid samples were collected by NASA’s OSIRIS-REx spacecraft, which flew by Earth early Sunday morning and jettisoned the capsule over a designated landing zone in the Utah desert. T

Split text from the article into chunks

In [220]:
sentences = []
for i, article in enumerate(articles):
  article = article.replace('.', '.<eos>')
  article = article.replace('!', '!<eos>')
  article = article.replace('?', '?<eos>')
  sentences.append(article.split('<eos>'))
sentences[-1]

['Since ChatGPT’s release in late 2022, many news outlets have reported on the ethical threats posed by artificial intelligence.',
 ' Tech pundits have issued warnings of killer robots bent on human extinction, while the World Economic Forum predicted that machines will take away jobs.',
 ' The tech sector is slashing its workforce even as it invests in AI-enhanced productivity tools.',
 ' Writers and actors in Hollywood are on strike to protect their jobs and their likenesses.',
 ' And scholars continue to show how these systems heighten existing biases or create meaningless jobs – amid myriad other problems.',
 ' There is a better way to bring artificial intelligence into workplaces.',
 ' I know, because I’ve seen it, as a sociologist who works with NASA’s robotic spacecraft teams.',
 ' The scientists and engineers I study are busy exploring the surface of Mars with the help of AI-equipped rovers.',
 ' But their job is no science fiction fantasy.',
 ' It’s an example of the power of 

In [229]:
max_tokens_chunk = 500
chunks = []

for i, article_sentences in enumerate(sentences):
  chunks.append([])
  current_chunk = 0
  for sentence in article_sentences:
    if len(chunks[i]) == current_chunk + 1:
      if len(chunks[i][current_chunk]) + len(sentence.split(' ')) <= max_tokens_chunk:
        chunks[current_chunk].extend(sentence.split(' '))
      else:
        current_chunk += 1
        chunks[i].append(sentence.split(' '))
    else:
      print(current_chunk)
      chunks[i].append(sentence.split(' '))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


In [231]:
chunks[0]

[['',
  'Profile',
  'Sections',
  'tv',
  'Featured',
  'More',
  'From',
  'NBC',
  'Follow',
  'NBC',
  'News',
  '‘I',
  'am',
  'not',
  'going',
  'anywhere’:',
  'Sen.'],
 '',
 'Bob',
 'Menendez',
 'rejects',
 'calls',
 'to',
 'resign',
 'over',
 'bribery',
 'charges',
 'September',
 '22,',
 '2023,',
 '4:39',
 'PM',
 'New',
 'Jersey',
 'Gov.',
 ['', 'Phil', 'Murphy', 'calls', 'on', 'Sen.'],
 ['',
  'Bob',
  'Menendez',
  'to',
  'resign',
  'over',
  'bribery',
  'charges',
  'September',
  '22,',
  '2023,',
  '3:07',
  'PM',
  'Biden',
  'tells',
  'Zelenskyy',
  'that',
  'the',
  'U.'],
 ['S.'],
 ['',
  'will',
  'send',
  'long-range',
  'missiles',
  'that',
  'Ukraine',
  'has',
  'repeatedly',
  'asked',
  'for,',
  'people',
  'familiar',
  'with',
  'the',
  'discussions',
  'say',
  'September',
  '22,',
  '2023,',
  '11:49',
  'AM',
  'Autoworkers',
  'strike',
  'will',
  'expand',
  'to',
  'more',
  'GM',
  'and',
  'Stellantis',
  'plants,',
  'and',
  'the',
  'h

In [206]:
joined_chunks = []

for i, article_chunks in enumerate(chunks):
    article_text = []

    for chunk_id in range(len(article_chunks)):
        article_text.append(' '.join(article_chunks[chunk_id]))

    joined_chunks.append(article_text)

In [187]:
joined_chunks[0][0]

' Profile Sections tv Featured More From NBC Follow NBC News ‘I am not going anywhere’: Sen.'